In [1]:
%%bash
wget https://huggingface.co/datasets/jwnt4/qvhighlights-val/resolve/main/highlight_val_release.jsonl

--2025-05-05 13:36:05--  https://huggingface.co/datasets/jwnt4/qvhighlights-val/resolve/main/highlight_val_release.jsonl
Resolving huggingface.co (huggingface.co)... 3.164.240.38, 3.164.240.43, 3.164.240.65, ...
Connecting to huggingface.co (huggingface.co)|3.164.240.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 821101 (802K) [text/plain]
Saving to: ‘highlight_val_release.jsonl’

     0K .......... .......... .......... .......... ..........  6% 39.7M 0s
    50K .......... .......... .......... .......... .......... 12% 30.7M 0s
   100K .......... .......... .......... .......... .......... 18% 74.9M 0s
   150K .......... .......... .......... .......... .......... 24% 66.0M 0s
   200K .......... .......... .......... .......... .......... 31% 84.0M 0s
   250K .......... .......... .......... .......... .......... 37% 91.3M 0s
   300K .......... .......... .......... .......... .......... 43% 24.3M 0s
   350K .......... .......... .......... .......... ..

In [2]:
import json
import numpy as np

In [23]:
with open('highlight_val_release.jsonl', 'r') as f_in:
    all_data = [line for line in f_in.readlines()]

In [24]:
len(all_data)

1550

In [25]:
all_data[0]

'{"qid": 2579, "query": "A girl and her mother cooked while talking with each other on facetime.", "duration": 150, "vid": "NUsG9BgSes0_210.0_360.0", "relevant_clip_ids": [41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74], "saliency_scores": [[1, 1, 2], [1, 1, 3], [1, 1, 3], [1, 1, 3], [1, 1, 3], [1, 1, 2], [1, 1, 3], [2, 1, 4], [2, 1, 4], [2, 1, 4], [2, 2, 4], [2, 2, 3], [3, 2, 2], [4, 2, 2], [4, 2, 2], [4, 2, 2], [3, 2, 2], [3, 2, 3], [3, 2, 3], [4, 2, 3], [3, 2, 3], [3, 2, 3], [3, 2, 3], [3, 2, 3], [3, 2, 2], [3, 2, 2], [3, 2, 2], [3, 2, 3], [3, 2, 4], [3, 2, 3], [3, 2, 4], [3, 2, 4], [3, 1, 3], [3, 1, 4]], "relevant_windows": [[82, 150]]}\n'

In [26]:
train = 1085
test = 232
val = 233

In [27]:
train + test + val

1550

In [31]:
train_data = all_data[:train]
test_data = all_data[train: train+test]
val_data = all_data[train+test:]
data = {
    "train": train_data,
    "test": test_data,
    "val": val_data
}
len(train_data), len(test_data), len(val_data)


(1085, 232, 233)

In [34]:
for split in ['train', 'test', 'val']:
    with open(f"datasets/qvhighlights/annotations/raw/highlights_{split}_release.jsonl", 'w') as f_out:
        f_out.writelines(data[split])

In [2]:
from huggingface_hub import hf_hub_download

In [37]:
hf_hub_download(
    repo_id="jwnt4/qvhighlights-val", repo_type='dataset',
    filename='videos.tar.gz', 
)

videos.tar.gz:   0%|          | 0.00/17.6G [00:00<?, ?B/s]

'/root/.cache/huggingface/hub/datasets--jwnt4--qvhighlights-val/snapshots/830b7416b72181aba579ff691440a34814f328ce/videos.tar.gz'

In [ ]:
!mv '/root/.cache/huggingface/hub/datasets--jwnt4--qvhighlights-val/snapshots/830b7416b72181aba579ff691440a34814f328ce/videos.tar.gz' "videos.tar.gz"

In [ ]:
print('e')

In [4]:
hf_hub_download(
    repo_id="jwnt4/qvhighlights-val", repo_type='dataset',
    filename='videos.tar.gz', local_dir="/workspace/"
)

videos.tar.gz:   0%|          | 0.00/17.6G [00:00<?, ?B/s]

'/workspace/videos.tar.gz'

In [ ]:
import json
import re

def json_dumps(data, indent=2, max_inline_length=120):
    json_str = json.dumps(data, indent=indent)
    # regex to find lists with no nested braces/brackets
    pattern = re.compile(r"\[\s*([^\[\]\{\}]+?)\s*\]", re.DOTALL)

    def replacer(match):
        content = match.group(1)
        # remove whitespace and newlines inside the list
        inline = " ".join(content.split())
        if len(inline) <= max_inline_length:
            return f"[ {inline} ]"
        else:
            return match.group(0)

    return pattern.sub(replacer, json_str)